# *psycopg2* を使うサンプル

*psycopg2* モジュールを使ってデータベースに接続します。テーブルを作成し、[ipython-sql](https://github.com/catherinedevlin/ipython-sql) の ``%%sql`` でクエリの結果を確認します。

Python モジュールを読み込み、環境変数から接続情報を取得します。

In [1]:
import os

import psycopg2

pgconfig = {
    'host': os.getenv('PGHOST'),
    'port': os.getenv('PGPORT', 5432),
    'database': os.getenv('PGDATABASE'),
    'user': os.getenv('PGUSER'),
    'password': os.getenv('PGPASSWORD'),
}

環境変数から取得した接続情報でデータベースに接続します。

In [2]:
conn = psycopg2.connect(**pgconfig)
conn

<connection object at 0x7f13d81a92b8; dsn: 'user=postgres password=xxx dbname=postgres host=postgres port=5432', closed: 0>

SQL を実行するためのカーソルを取得します。

In [3]:
cur = conn.cursor()

テーブルを定義します。

In [4]:
cur.execute('''
DROP TABLE IF EXISTS t ;

CREATE TABLE t (
    id SERIAL,
    code CHAR(2) NOT NULL,
    name TEXT NOT NULL,
    created_at TIMESTAMP WITHOUT TIME ZONE DEFAULT now()
)''')

データを登録します。

In [5]:
cur.execute('''
INSERT INTO t (
    code, name
) VALUES
    ('01', '北海道'),
    ('47', '沖縄県')
''')

データを取得します。

In [6]:
cur.execute("SELECT * FROM t")
print(cur.fetchone())
print(cur.fetchone())

(1, '01', '北海道', datetime.datetime(2020, 4, 17, 14, 49, 29, 996229))
(2, '47', '沖縄県', datetime.datetime(2020, 4, 17, 14, 49, 29, 996229))


カーソルを閉じます。

In [7]:
cur.close()

登録したデータをコミットしてから接続を閉じます。

In [8]:
conn.commit()
conn.close()

`%%sql` コマンドを使ってデータ登録内容を確認します。接続情報は環境変数から読み込みますので、接続文字列は *postgresql://* だけで認識してくれます。

In [9]:
%load_ext sql

In [10]:
%%sql postgresql://
select * from t;

2 rows affected.


id,code,name,created_at
1,01,北海道,2020-04-17 14:49:29.996229
2,47,沖縄県,2020-04-17 14:49:29.996229
